# Test Parallel ExIFFI

In [38]:
import sys
import numpy as np
import pandas as pd
#from tqdm import tqdm
from append_dir import append_dirname
append_dirname('ExIFFI')
from utils.utils import partition_data
from utils.feature_selection import *
#from plot import *
#from simulation_setup import *
from models import *
from models.Extended_IF import *
from models.Extended_DIFFI_parallel import *
from models.Extended_DIFFI_original import *
import math
import seaborn as sns
sns.set()

from sklearn.preprocessing import StandardScaler
import time

import os
import pickle 
from scipy.io import loadmat
from glob import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

## Utility Functions

In [39]:
path=os.getcwd()
path=os.path.dirname(path)
path_real=os.path.join(path,'data','real')
path_syn=os.path.join(path,'data','synthetic')
mat_files_real=glob(os.path.join(path_real, '*.mat'))
mat_file_names_real={os.path.basename(x).split('.')[0]:x for x in mat_files_real}
files_syn=glob(os.path.join(path_syn, '*.pkl'))
file_names_syn={os.path.basename(x).split('.')[0]:x for x in files_syn}
csv_files=glob(os.path.join(path, '*.csv'))
csv_file_names={os.path.basename(x).split('.')[0]:x for x in csv_files}

In [40]:
def load_data_syn(filename,path):
    file_to_read = open(os.path.join(path,filename), "rb")
    dict = pickle.load(file_to_read)
    data=[]
    for k in dict.keys():
        data.append(dict[k])

    return data

In [41]:
file_to_read = open(os.path.join(path_syn,'ball_6_dim.pkl'), "rb")
loaded_dictionary = pickle.load(file_to_read)
X_train=loaded_dictionary['X_train']
file_to_read = open(os.path.join(path_syn,'anomalies.pkl'), "rb")
loaded_dictionary = pickle.load(file_to_read)
X_xaxis,X_yaxis,X_bisect,X_bisect_3d,X_bisect_6d=loaded_dictionary['X_xaxis'],loaded_dictionary['X_yaxis'],loaded_dictionary['X_bisec'],loaded_dictionary['X_bisec_3d'],loaded_dictionary['X_bisec_6d']
file_to_read = open(os.path.join(path_syn,'toy_datasets.pkl'), "rb")
loaded_dictionary = pickle.load(file_to_read)
X_toy_2d,X_toy_3d,X_toy_4d,X_toy_6d,y = loaded_dictionary['X_toy_2d'],loaded_dictionary['X_toy_3d'],loaded_dictionary['X_toy_4d'],loaded_dictionary['X_toy_6d'],loaded_dictionary['y']

In [42]:
def load_data(filename):
    data=loadmat(mat_file_names_real[filename])
    X,y=data['X'],data['y']
    y=np.hstack(y)
    return X,y 

def load_data_csv(filename):
    data=pd.read_csv(csv_file_names[filename])
    if 'Unnamed: 0' in data.columns:
        data=data.drop(columns=['Unnamed: 0'])
    X=data[data.columns[data.columns!='Target']]
    y=data['Target']
    return X,y

In [43]:
def compute_imps(model,X_train,X_test,n_runs,name,pwd,dim,f=6):

    name='GFI_'+name

    #X_test=np.r_[X_train,X_test]

    imps=np.zeros(shape=(n_runs,X_train.shape[1]))
    for i in tqdm(range(n_runs)):
        model.fit(X_train)
        imps[i,:]=model.Global_importance(X_test,calculate=True,overwrite=False,depth_based=False)

    path = pwd + '/results/imp/imp_score_' + name + '.pkl'
    with open(path, 'wb') as fl:
        pickle.dump(imps,fl)

    return imps

In [44]:
def interpretation_plots(name,pwd):

    os.chdir('c:\\Users\\lemeda98\\Desktop\\PHD Information Engineering\\ExIFFI\\ExIFFI\\data')
    if name=='diabetes' or name=='moodify':
        X,y=csv_dataset(name,os.getcwd()+'\\')
    else:
        X,y=dataset(name,os.getcwd()+'\\')

    X,y=downsample(X,y)
    X_train,X_test=partition_data(X,y)
    scaler=StandardScaler()
    X_train=scaler.fit_transform(X_train)
    X_test=scaler.transform(X_test)
    y_train=np.zeros(X_train.shape[0])
    y_test=np.ones(X_test.shape[0])
    y=np.concatenate([y_train,y_test])
    X_test=np.r_[X_train,X_test]
    scaler2=StandardScaler()
    X=scaler2.fit_transform(X)
    EDIFFI=Extended_DIFFI_original(300,max_depth=100,subsample_size=256,plus=1)
    dim=X.shape[1]

    # No Split 

    imps,plt_data=compute_imps(EDIFFI,X,X,10,name,pwd,dim,f=6)

    path = pwd + '\\results\\davide\\Importance_Scores\\Imp Score\\imp_score_GFI_' + name + '.pkl'
    with open(path, 'rb') as fl:
        imps = pickle.load(fl)

## Load Data

## Wine Dataset

In [45]:
name='wine'
X,y=load_data(name)
X_train,X_test=partition_data(X,y)
X.shape,y.shape

((129, 13), (129,))

In [46]:
X_train.shape,X_test.shape

((119, 13), (10, 13))

In [22]:
EDIFFI=Extended_DIFFI_original(300,max_depth=100,subsample_size=256,plus=1,seed=126)

### Serial ExIFFI

In [7]:
from sklearn.preprocessing import StandardScaler
import time
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
y_train=np.zeros(X_train.shape[0])
y_test=np.ones(X_test.shape[0])
y=np.concatenate([y_train,y_test])
X_test=np.r_[X_train,X_test]
scaler2=StandardScaler()
X=scaler2.fit_transform(X)
EDIFFI=Extended_DIFFI_original(300,max_depth=100,subsample_size=256,plus=1)
dim=X.shape[1]
pwd=os.path.dirname(os.getcwd())
start=time.time()
imps,plt_data=compute_imps(EDIFFI,X,X,10,name,pwd,dim,f=6)
end=time.time()
print(f'Elapsed time: {end-start}')

### Parallel ExIFFI

In [8]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
y_train=np.zeros(X_train.shape[0])
y_test=np.ones(X_test.shape[0])
y=np.concatenate([y_train,y_test])
X_test=np.r_[X_train,X_test]
scaler2=StandardScaler()
X=scaler2.fit_transform(X)
EDIFFI=Extended_DIFFI_parallel(300,max_depth=100,subsample_size=256,plus=1)
EDIFFI.set_num_processes(8)

dim=X.shape[1]
pwd=os.path.dirname(os.getcwd())


start=time.time()
imps,plt_data=compute_imps(EDIFFI,X,X,10,name,pwd,dim,f=6)
end=time.time()

print(f'Elapsed time: {end-start}')

## Ionosphere Dataset

In [5]:
name='ionosphere'
X,y=load_data(name)
X_train,X_test=partition_data(X,y)
X.shape,y.shape

((351, 33), (351,))

### Serial ExIFFI

In [19]:
from sklearn.preprocessing import StandardScaler
import time
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
y_train=np.zeros(X_train.shape[0])
y_test=np.ones(X_test.shape[0])
y=np.concatenate([y_train,y_test])
X_test=np.r_[X_train,X_test]
scaler2=StandardScaler()
X=scaler2.fit_transform(X)
EDIFFI=Extended_DIFFI_original(300,max_depth=100,subsample_size=256,plus=1)
dim=X.shape[1]
pwd=os.path.dirname(os.getcwd())
start=time.time()
imps,plt_data=compute_imps(EDIFFI,X,X,10,name,pwd,dim,f=6)
end=time.time()
print(f'Elapsed time: {end-start}')

100%|██████████| 10/10 [02:54<00:00, 17.46s/it]

Elapsed time: 174.5880789756775


### Parallel ExIFFI

In [6]:
from sklearn.preprocessing import StandardScaler
import time
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
y_train=np.zeros(X_train.shape[0])
y_test=np.ones(X_test.shape[0])
y=np.concatenate([y_train,y_test])
X_test=np.r_[X_train,X_test]
scaler2=StandardScaler()
X=scaler2.fit_transform(X)
EDIFFI=Extended_DIFFI_parallel(300,max_depth=100,subsample_size=256,plus=1)
EDIFFI.set_num_processes(12)

dim=X.shape[1]
pwd=os.path.dirname(os.getcwd())


start=time.time()
imps,plt_data=compute_imps(EDIFFI,X,X,10,name,pwd,dim,f=6)
end=time.time()

print(f'Elapsed time: {end-start}')

100%|██████████| 10/10 [01:29<00:00,  8.92s/it]

Elapsed time: 89.17974472045898


#### Results Ionosphere

Serial -> 174.5880789756775

8 threads

Parallel -> 147.8202040195465

12 threads

Parallel -> 137.81197714805603

In [16]:
from sklearn.preprocessing import StandardScaler
import time
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
y_train=np.zeros(X_train.shape[0])
y_test=np.ones(X_test.shape[0])
y=np.concatenate([y_train,y_test])
X_test=np.r_[X_train,X_test]
scaler2=StandardScaler()
X=scaler2.fit_transform(X)
EDIFFI=Extended_DIFFI_parallel(300,max_depth=100,subsample_size=256,plus=1)

In [17]:
EDIFFI.fit(X_train)

## Diabetes Dataset

In [23]:
X,y=load_data_csv('diabetes')
X.shape,y.shape

((100000, 4), (100000,))

In [35]:
stats=np.load('test_stat.npz',allow_pickle=True)

In [32]:
stats['execution_time_stat']

array({'mean': 3.6890564918518067, 'std': 0.5207048001277851},
      dtype=object)

In [36]:
stats['importances_matrix']

array({'Execution 0': array([[3.44755852, 4.13613304, 3.37304616, 3.90623504, 3.63012237,
        3.30479825, 3.527404  , 4.06928892, 3.49031055, 3.56519344,
        3.38879328, 3.45615214, 3.9789097 ],
       [3.44755852, 4.13613304, 3.37304616, 3.90623504, 3.63012237,
        3.30479825, 3.527404  , 4.06928892, 3.49031055, 3.56519344,
        3.38879328, 3.45615214, 3.9789097 ],
       [3.44755852, 4.13613304, 3.37304616, 3.90623504, 3.63012237,
        3.30479825, 3.527404  , 4.06928892, 3.49031055, 3.56519344,
        3.38879328, 3.45615214, 3.9789097 ],
       [3.44755852, 4.13613304, 3.37304616, 3.90623504, 3.63012237,
        3.30479825, 3.527404  , 4.06928892, 3.49031055, 3.56519344,
        3.38879328, 3.45615214, 3.9789097 ],
       [3.44755852, 4.13613304, 3.37304616, 3.90623504, 3.63012237,
        3.30479825, 3.527404  , 4.06928892, 3.49031055, 3.56519344,
        3.38879328, 3.45615214, 3.9789097 ],
       [3.44755852, 4.13613304, 3.37304616, 3.90623504, 3.63012237,
     